# Open Channel Flow Calculations

## This notebook shows how to calculate some common Open Channel Flow calculations such as:

1. $Q$ from $y_n$ (and channel properties) using either Manning's or Chezy equation
2. $y_n$ from $Q$ and other channel properties
3. Critical depth, $y_c$
4. Critical Slope, $So_c$
5. Solution of cubic equations e.g. for *Specific Energy* calculations 


## Chezy form of the Uniform Flow equation
$$
Q = ACR^{1/2}S_o^{1/2}
$$

## Manning's form of the Uniform Flow equation
$$
Q = \frac{1}{n} AR^{2/3}S_o^{1/2}
$$

## Chezy example $Q$ from $y_n$ for rectangular channel


Here are the equations for properties of a rectangular channel given the depth $y$
    
If flow in a $6m$ wide  rectangular channel is $4m$ deep running when in uniform flow what wiil be the discharge, $Q$ if the Chezy $C = 50 m^{1/2}/s$ and the slope of the channel is 1 in 1000?

We will need these equations of the channel properties:

![Properties of a rectangular channel](./images/channel_props_rectangular.png "Properties of a rectangular channel")



In [53]:
b = 6
C = 50
yn = 4
slope_v = 1
slope_h = 1000
So = slope_v/slope_h

A = b*yn
P = b + 2*yn
R = A/P

Q = A*C*(R**0.5)*(So**0.5)

print("Q = %6.3f m^3/s" % (Q))

Q = 49.685 m^3/s


## Manning's example $Q$ from $y_n$ for rectangular channel

Same example as above but use Manning's $n = 0.02$


In [54]:
b = 6
n = 0.02
yn = 4
slope_v = 1
slope_h = 1000
So = slope_v/slope_h

A = b*yn
P = b + 2*yn
R = A/P

Q = A*(R**(2/3))*(So**0.5)/n

print("Q = %6.3f m^3/s" % (Q))

Q = 54.355 m^3/s


## Chezy equation calculate *normal depth*, $y_n$, from $Q$ and channel properties

For a rectangular channel of width $6m$, flowing in *uniform flow* with $Q = 30m^3/s$ calculate the normal depth $y_n$
if the Chezy $C = 50 m^{1/2}/s$ and the channel slopes with a drop of $1m$ in $1000m$

In [55]:
def secant_yn_rect_chezy(y1, y2, b, C, So, Q, debug):
    tol = 0.0005
    max_itts = 20

    dy = 10.0*tol

    for i in range(max_itts):

        A1 = b*y1
        P1 = b + 2*y1
        R1 = A1/P1
        Q1 = A1*C*(R1**(0.5))*(So**(0.5))
        f1 = Q-Q1
    
        A2 = b*y2
        P2 = b + 2*y2
        R2 = A2/P2
        Q2 = A2*C*(R2**(0.5))*(So**(0.5))
        f2 = Q-Q2

        dy = f2*(y2-y1)/(f2-f1)
        y_new = y2 - dy
        
        if(debug == 1):
            print("%d y1 = %6.3f m f1 = %6.3f m3/s   y2 = %6.3f m f2 = %6.3f m3/s dy = %6.4f" % (i,y1,f1,y2,f2,abs(dy)))
            

        if(abs(dy) < tol):
            y2 = y_new
            break
            
        y1 = y2
        y2 = y_new

    if(abs(dy) >= tol):
        print("WARNING: The secant method did not converge after %d iterations." % (max_itts) )

    yn = y2
    return yn

Q = 30
b = 6
C = 50
slope_v = 1
slope_h = 1000
So = slope_v/slope_h

debug = 1

y1 = 1
y2 = 2
yn = secant_yn_rect_chezy(y1, y2, b, C, So, Q, debug)

print("yn = %6.3f m" % (yn))

0 y1 =  1.000 m f1 = 21.784 m3/s   y2 =  2.000 m f2 =  9.215 m3/s dy = 0.7332
1 y1 =  2.000 m f1 =  9.215 m3/s   y2 =  2.733 m f2 = -1.009 m3/s dy = 0.0724
2 y1 =  2.733 m f1 = -1.009 m3/s   y2 =  2.661 m f2 =  0.024 m3/s dy = 0.0017
3 y1 =  2.661 m f1 =  0.024 m3/s   y2 =  2.663 m f2 =  0.000 m3/s dy = 0.0000
yn =  2.663 m


## Chezy equation to calculate *normal depth*,  $y_n$ for a Trapezoidal Channel

 <!--![Properties of a Trapezoidal channel](./images/channel_props_trap.png|width="100")-->

Use these formulae for the channel properties

<img src="./images/channel_props_trap.png" width="600">

In this example calculate the normal depth for a channel with this geometry and discharge:

$Q = 4.5m^3/s$, Bed width $b = 3.0m$, Side slope $s = 3$, Bed slope $1:100$, Chezy $C = 80 m^{1/2}/s$

In [56]:
import math
def secant_yn_trap_chezy(y1, y2, b, s, C, So, Q, debug):
    tol = 0.0005
    max_itts = 20

    dy = 10.0*tol

    for i in range(max_itts):

        A1 = (b + s*y1) * y1
        P1 = b + 2 * y1*math.sqrt(1+s*s)
        R1 = A1 / P1
        Q1 = A1 * C * (R1 ** (0.5)) * (So ** (0.5))
        f1 = Q - Q1
        if (debug == 2):
            print("%d A1 = %6.3f m2 P1 = %6.3f m   R1 = %6.3f m Q1 = %6.3f m3/s f1 = %6.4f" % (i, A1, P1, R1, Q1, f1))
        A2 = (b + s*y2) * y2
        P2 = b + 2 * y2*math.sqrt(1+s*s)
        R2 = A2 / P2
        Q2 = A2 * C * (R2 ** (0.5)) * (So ** (0.5))
        f2 = Q - Q2
        if (debug == 2):
            print("%d A2 = %6.3f m2 P2 = %6.3f m   R2 = %6.3f m Q2 = %6.3f m3/s f2 = %6.4f" % (i, A2, P2, R2, Q2, f2))

        dy = f2 * (y2 - y1) / (f2 - f1)
        y_new = y2 - dy

        if (debug > 0):
            print("%d y1 = %6.3f m f1 = %6.3f m3/s   y2 = %6.3f m f2 = %6.3f m3/s dy = %6.4f" % (i, y1, f1, y2, f2, abs(dy)))

        if (abs(dy) < tol):
            y2 = y_new
            break

        y1 = y2
        y2 = y_new

    if(abs(dy) >= tol):
        print("WARNING: The secant method did not converge after %d iterations." % (max_itts) )

    yn = y2
    return yn

Q = 4.5
b = 4
s = 3
C = 80
slope_v = 1
slope_h = 100

So = slope_v / slope_h

debug = 0

y1 = 1
y2 = 2
yn = secant_yn_trap_chezy(y1, y2, b, s, C, So, Q, debug)

print("yn = %6.3f m" % (yn))


yn =  0.254 m


## Manning's equation to calculate *normal depth*,  $y_n$ for a Trapezoidal Channel

In this example calculate the normal depth for a channel with this geometry and discharge:

$Q =30.0m^3/s$, Bed width $b = 4.0m$, Side slope $s = 2.5$, Bed slope $6.7m$ height drop over $10km$, Manning's $n = 0.015$

In [57]:
import math
def secant_yn_trap_manning(y1, y2, b, s, n, So, Q, debug):
    tol = 0.0005
    max_itts = 20

    dy = 10.0*tol

    for i in range(max_itts):

        A1 = (b + s*y1) * y1
        P1 = b + 2 * y1*math.sqrt(1+s*s)
        R1 = A1 / P1
        Q1 = A1 * (R1 ** (2/3)) * (So ** (0.5))/n
        f1 = Q - Q1
        if (debug == 2):
            print("%d A1 = %6.3f m2 P1 = %6.3f m   R1 = %6.3f m Q1 = %6.3f m3/s f1 = %6.4f" % (i, A1, P1, R1, Q1, f1))
        A2 = (b + s*y2) * y2
        P2 = b + 2 * y2*math.sqrt(1+s*s)
        R2 = A2 / P2
        Q2 = A2 *  (R2 ** (2/3)) * (So ** (0.5))/n
        f2 = Q - Q2
        if (debug == 2):
            print("%d A2 = %6.3f m2 P2 = %6.3f m   R2 = %6.3f m Q2 = %6.3f m3/s f2 = %6.4f" % (i, A2, P2, R2, Q2, f2))

        dy = f2 * (y2 - y1) / (f2 - f1)
        y_new = y2 - dy

        if (debug > 0):
            print("%d y1 = %6.3f m f1 = %6.3f m3/s   y2 = %6.3f m f2 = %6.3f m3/s dy = %6.4f" % (i, y1, f1, y2, f2, abs(dy)))

        if (abs(dy) < tol):
            y2 = y_new
            break

        y1 = y2
        y2 = y_new

    if(abs(dy) >= tol):
        print("WARNING: The secant method did not converge after %d iterations." % (max_itts) )

    yn = y2
    return yn


Q = 30
b = 4
s = 2.5
n = 0.015
slope_v = 6.7
slope_h = 10000

So = slope_v / slope_h

debug = 1

y1 = 1
y2 = 2
yn = secant_yn_trap_manning(y1, y2, b, s, n, So, Q, debug)

print("yn = %6.3f m" % (yn))


0 y1 =  1.000 m f1 = 21.220 m3/s   y2 =  2.000 m f2 = -5.438 m3/s dy = 0.2040
1 y1 =  2.000 m f1 = -5.438 m3/s   y2 =  1.796 m f2 =  1.702 m3/s dy = 0.0486
2 y1 =  1.796 m f1 =  1.702 m3/s   y2 =  1.845 m f2 =  0.085 m3/s dy = 0.0026
3 y1 =  1.845 m f1 =  0.085 m3/s   y2 =  1.847 m f2 = -0.001 m3/s dy = 0.0000
yn =  1.847 m


## Manning's equation to calculate *normal depth*,  $y_n$ for a Circular Channel

Use these formulae for the channel properties

<img src="./images/channel_props_circular.png" width="600">

In this example calculate the normal depth for a channel with this geometry and discharge:

$Q = 4.5m^3/s$, Bed width $b = 3.0m$, Side slope $s = 3$, Bed slope $1:100$, Chezy $C = 80 m^{1/2}/s$

In [58]:
import math
def secant_yn_circular_manning(y1, y2, D, n, So, Q, debug):
    tol = 0.0005
    max_itts = 20

    if (debug == 2):
        print("So = %6.3f   D = %6.3f m  n = %6.5f" % (So,D,n))

    dy = 10.0*tol

    for i in range(max_itts):

        phi1 = 2*math.acos(1-2*y1/D)
        A1 = (phi1 - math.sin(phi1)) * D*D/8
        P1 = phi1*D/2
        R1 = A1 / P1
        Q1 = A1 * (R1 ** (2/3)) * (So ** (0.5))/n
        f1 = Q - Q1
        if (debug == 2):
            print("%d phi1 = %6.3f rads  y1 = %6.3f m" % (i, phi1, y1))
            print("%d A1 = %6.3f m2 P1 = %6.3f m   R1 = %6.3f m Q1 = %6.3f m3/s f1 = %6.4f" % (i, A1, P1, R1, Q1, f1))

        phi2 = 2*math.acos(1-2*y2/D)
        A2 = (phi2 - math.sin(phi2)) * D*D/8
        P2 = phi2*D/2
        R2 = A2 / P2
        Q2 = A2 *  (R2 ** (2/3)) * (So ** (0.5))/n
        f2 = Q - Q2
        if (debug == 2):
            print("%d phi2 = %6.3f rads  y2 = %6.3f m" % (i, phi2, y2))
            print("%d A2 = %6.3f m2 P2 = %6.3f m   R2 = %6.3f m Q2 = %6.3f m3/s f2 = %6.4f" % (i, A2, P2, R2, Q2, f2))

        dy = f2 * (y2 - y1) / (f2 - f1)
        y_new = y2 - dy

        if (debug > 0):
            print("%d y1 = %6.3f m f1 = %6.3f m3/s   y2 = %6.3f m f2 = %6.3f m3/s dy = %6.4f" % (i, y1, f1, y2, f2, abs(dy)))

        if (abs(dy) < tol):
            y2 = y_new
            break

        y1 = y2
        y2 = y_new

    if(abs(dy) >= tol):
        print("WARNING: The secant method did not converge after %d iterations." % (max_itts) )

    yn = y2
    return yn

Q = 2.0
D = 5
n = 0.014
slope_v = 1.0
slope_h = 50

So = slope_v / slope_h

debug = 1

y1 = D/4
y2 = D/2
yn = secant_yn_circular_manning(y1, y2, D, n, So, Q, debug)

print("yn = %6.3f m" % (yn))

0 y1 =  1.250 m f1 = -29.527 m3/s   y2 =  2.500 m f2 = -113.078 m3/s dy = 1.6918
1 y1 =  2.500 m f1 = -113.078 m3/s   y2 =  0.808 m f2 = -11.052 m3/s dy = 0.1833
2 y1 =  0.808 m f1 = -11.052 m3/s   y2 =  0.625 m f2 = -5.665 m3/s dy = 0.1927
3 y1 =  0.625 m f1 = -5.665 m3/s   y2 =  0.432 m f2 = -1.537 m3/s dy = 0.0718
4 y1 =  0.432 m f1 = -1.537 m3/s   y2 =  0.361 m f2 = -0.409 m3/s dy = 0.0260
5 y1 =  0.361 m f1 = -0.409 m3/s   y2 =  0.335 m f2 = -0.055 m3/s dy = 0.0040
6 y1 =  0.335 m f1 = -0.055 m3/s   y2 =  0.330 m f2 = -0.003 m3/s dy = 0.0002
yn =  0.330 m


# Critical Depth Calculations

*Critical depth* is the depth when the *Froude number*, $Fr$ is equal to 1.
$$Fr = \frac{V}{\sqrt{gD_m}} = \frac{Q}{A\sqrt{g\frac{A}{B}}}  $$
where $B$ is the surface width

So critical flow is $Fr=1$ and also $Fr^2 = 1$
$$Fr = \frac{Q}{A\sqrt{g\frac{A}{B}}}  = 1$$
and
$$Fr^2 = \frac{V^2}{gD_m} = \frac{QB}{A^3g} = 1$$

In this example, calcuate *critical depth* $y_c$ for the discharge $Q = 30m^3/s$ in a trapezoidal channel, bottom width $b = 4m$, side-slope $s = 2.5$

In [59]:
import math
def secant_yc_trap_manning(y1, y2, b, s, Q, debug):
    tol = 0.0005
    max_itts = 20

    g= 9.81

    dy = 10.0*tol

    for i in range(max_itts):

        A1 = (b + s*y1) * y1
        B1 = b + 2*s*y1
        Fr1 = Q / A1 / math.sqrt(g * A1/B1)
        f1 = 1 - Fr1
        if (debug == 2):
            print("%d A1 = %6.3f m2 B1 = %6.3f m   Fr1 = %6.3f m f1 = %6.4f" % (i, A1, B1, Fr1, f1))
        A2 = (b + s*y2) * y2
        B2 = b + 2*s*y2
        Fr2 = Q / A2 / math.sqrt(g * A2/B2)
        f2 = 1 - Fr2
        if (debug == 2):
            print("%d A2 = %6.3f m2 B2 = %6.3f m   Fr2 = %6.3f m f2 = %6.4f" % (i, A2, B2, Fr2, f2))

        dy = f2 * (y2 - y1) / (f2 - f1)
        y_new = y2 - dy

        if (debug > 0):
            print("%d y1 = %6.3f m f1 = %6.3f m3/s   y2 = %6.3f m f2 = %6.3f m3/s dy = %6.4f" % (i, y1, f1, y2, f2, abs(dy)))

        if (abs(dy) < tol):
            y2 = y_new
            break

        y1 = y2
        y2 = y_new

    if(abs(dy) >= tol):
        print("WARNING: The secant method did not converge after %d iterations." % (max_itts) )

    yc = y2
    return yc


Q = 30.0 # m3/s
b = 4.0 # m
s = 2.5 # no units

debug = 1

y1 = 1.0 # m
y2 = 3.0 # m
yc = secant_yc_trap_manning(y1, y2, b, s, Q, debug)


print("yc = %6.3f m" % (yc))


0 y1 =  1.000 m f1 = -0.734 m3/s   y2 =  3.000 m f2 =  0.794 m3/s dy = 1.0393
1 y1 =  3.000 m f1 =  0.794 m3/s   y2 =  1.961 m f2 =  0.512 m3/s dy = 1.8869
2 y1 =  1.961 m f1 =  0.512 m3/s   y2 =  0.074 m f2 = -115.549 m3/s dy = 1.8786
3 y1 =  0.074 m f1 = -115.549 m3/s   y2 =  1.952 m f2 =  0.508 m3/s dy = 0.0082
4 y1 =  1.952 m f1 =  0.508 m3/s   y2 =  1.944 m f2 =  0.504 m3/s dy = 1.0086
5 y1 =  1.944 m f1 =  0.504 m3/s   y2 =  0.936 m f2 = -0.954 m3/s dy = 0.6599
6 y1 =  0.936 m f1 = -0.954 m3/s   y2 =  1.596 m f2 =  0.272 m3/s dy = 0.1463
7 y1 =  1.596 m f1 =  0.272 m3/s   y2 =  1.449 m f2 =  0.125 m3/s dy = 0.1253
8 y1 =  1.449 m f1 =  0.125 m3/s   y2 =  1.324 m f2 = -0.037 m3/s dy = 0.0284
9 y1 =  1.324 m f1 = -0.037 m3/s   y2 =  1.352 m f2 =  0.004 m3/s dy = 0.0025
10 y1 =  1.352 m f1 =  0.004 m3/s   y2 =  1.350 m f2 =  0.000 m3/s dy = 0.0001
yc =  1.350 m
